In [ ]:
!pip install spacy



In [ ]:
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import spacy, json, random
from spacy.tokens import DocBin

with open("/content/drive/MyDrive/NER_Project/annotations.json", "r", encoding="utf-8") as f:
    data = json.load(f)["annotations"]

nlp = spacy.blank("en")
random.shuffle(data)
split = int(0.8 * len(data))
train, dev = data[:split], data[split:]

def to_docbin(dataset):
    db = DocBin()
    for text, ann in dataset:
        doc = nlp.make_doc(text)
        doc.ents = [doc.char_span(s, e, l) for s, e, l in ann["entities"] if doc.char_span(s, e, l)]
        db.add(doc)
    return db

to_docbin(train).to_disk("/content/drive/MyDrive/NER_Project/train.spacy")
to_docbin(dev).to_disk("/content/drive/MyDrive/NER_Project/dev.spacy")

print("Files saved in Drive")


Files saved in Drive


In [ ]:
!python -m spacy init fill-config "/content/drive/MyDrive/NER_Project/base_config.cfg" "/content/drive/MyDrive/NER_Project/config.cfg"


✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/NER_Project/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
config_path = "/content/drive/MyDrive/NER_Project/base_config.cfg"
print("Config file path:", config_path)

Config file path: /content/drive/MyDrive/NER_Project/base_config.cfg


In [ ]:
!ls "/content/drive/MyDrive/NER_Project"


 annotations.json   config.cfg	'Finance data.txt'
 base_config.cfg    dev.spacy	 train.spacy


In [ ]:
!python -m spacy train "/content/drive/MyDrive/NER_Project/config.cfg" \
  --output "/content/drive/MyDrive/NER_Project/output" \
  --paths.train "/content/drive/MyDrive/NER_Project/train.spacy" \
  --paths.dev "/content/drive/MyDrive/NER_Project/dev.spacy"


ℹ Saving to output directory:
/content/drive/MyDrive/NER_Project/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     49.70    0.00    0.00    0.00    0.00
200     200          6.73    843.02    0.00    0.00    0.00    0.00
400     400          0.00      0.00    0.00    0.00    0.00    0.00
600     600          0.00      0.00    0.00    0.00    0.00    0.00
800     800          0.00      0.00    0.00    0.00    0.00    0.00
1000    1000          0.00      0.00    0.00    0.00    0.00    0.00
1200    1200          0.00      0.00    0.00    0.00    0.00    0.00
1400    1400          0.00      0.00    0.00    0.00    0.00  

In [ ]:
!ls "/content/drive/MyDrive/NER_Project/output"



model-best  model-last


In [ ]:
import spacy
nlp = spacy.load("/content/drive/MyDrive/NER_Project/output/model-best")
text = """Acme Corp reported a quarterly revenue of $5.2 million in Q1 2024, showing strong growth.
Its stock ticker ACME rose by 12% on the New York Stock Exchange after the announcement.
The company also announced a merger with Global Finance Ltd to expand its overseas presence.
Meanwhile, investors are closely watching the Federal Reserve’s decision on interest rates.
Bitcoin surged past $40,000, while Ethereum traded around $2,500 during the same week.
Financial analysts predicted that tech sector earnings will outperform the broader S&P 500 index.
Overall, the finance market remains optimistic despite inflation concerns in the United States."""

doc = nlp(text)
print("Entities, Labels, and Positions:")
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}, Start: {ent.start_char}, End: {ent.end_char}")


Entities, Labels, and Positions:
Entity: Acme Corp, Label: COMPANY, Start: 0, End: 9
Entity: $5.2 million, Label: MONEY, Start: 42, End: 54
Entity: Q1 2024, Label: TIME, Start: 58, End: 65
Entity: ACME rose, Label: SECTOR, Start: 109, End: 118
Entity: York Stock Exchange, Label: COMPANY, Start: 137, End: 156
Entity: Global Finance Ltd, Label: COMPANY, Start: 224, End: 242
Entity: Federal Reserve’s, Label: ORG, Start: 324, End: 341
Entity: Bitcoin surged, Label: SECTOR, Start: 372, End: 386
Entity: $40,000,, Label: MONEY, Start: 392, End: 400
Entity: Ethereum traded, Label: SECTOR, Start: 407, End: 422
Entity: $2,500 during, Label: MONEY, Start: 430, End: 443
Entity: tech sector, Label: SECTOR, Start: 495, End: 506
Entity: S&P 500, Label: INDEX, Start: 544, End: 551
Entity: United States., Label: LOCATION, Start: 642, End: 656


In [ ]:
import spacy
nlp = spacy.load("/content/drive/MyDrive/NER_Project/output/model-last")
text = """Acme Corp reported a quarterly revenue of $5.2 million in Q1 2024, showing strong growth.
Its stock ticker ACME rose by 12% on the New York Stock Exchange after the announcement.
The company also announced a merger with Global Finance Ltd to expand its overseas presence.
Meanwhile, investors are closely watching the Federal Reserve’s decision on interest rates.
Bitcoin surged past $40,000, while Ethereum traded around $2,500 during the same week.
Financial analysts predicted that tech sector earnings will outperform the broader S&P 500 index.
Overall, the finance market remains optimistic despite inflation concerns in the United States."""

doc = nlp(text)
print("Entities, Labels, and Positions:")
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}, Start: {ent.start_char}, End: {ent.end_char}")


Entities, Labels, and Positions:
Entity: Acme Corp, Label: COMPANY, Start: 0, End: 9
Entity: $5.2 million, Label: MONEY, Start: 42, End: 54
Entity: Q1 2024, Label: TIME, Start: 58, End: 65
Entity: ACME rose, Label: SECTOR, Start: 109, End: 118
Entity: York Stock Exchange, Label: COMPANY, Start: 137, End: 156
Entity: Global Finance Ltd, Label: COMPANY, Start: 224, End: 242
Entity: Federal Reserve’s, Label: ORG, Start: 324, End: 341
Entity: Bitcoin surged, Label: SECTOR, Start: 372, End: 386
Entity: $40,000,, Label: MONEY, Start: 392, End: 400
Entity: Ethereum traded, Label: SECTOR, Start: 407, End: 422
Entity: $2,500 during, Label: MONEY, Start: 430, End: 443
Entity: tech sector, Label: SECTOR, Start: 495, End: 506
Entity: S&P 500, Label: INDEX, Start: 544, End: 551
Entity: United States., Label: LOCATION, Start: 642, End: 656


In [ ]:
!python -m spacy evaluate /content/drive/MyDrive/NER_Project/output/model-best /content/drive/MyDrive/NER_Project/dev.spacy


ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   0.00  
NER R   0.00  
NER F   0.00  
SPEED   1316  


=============================== NER (per type) ===============================

              P      R      F
MONEY      0.00   0.00   0.00
SECTOR     0.00   0.00   0.00
CRYPTO     0.00   0.00   0.00
TIME       0.00   0.00   0.00
COMPANY    0.00   0.00   0.00
PERCENT    0.00   0.00   0.00
TICKER     0.00   0.00   0.00
EXCHANGE   0.00   0.00   0.00



In [ ]:
!python -m spacy evaluate /content/drive/MyDrive/NER_Project/output/model-last /content/drive/MyDrive/NER_Project/dev.spacy


ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   0.00  
NER R   0.00  
NER F   0.00  
SPEED   847   


=============================== NER (per type) ===============================

              P      R      F
MONEY      0.00   0.00   0.00
SECTOR     0.00   0.00   0.00
CRYPTO     0.00   0.00   0.00
TIME       0.00   0.00   0.00
COMPANY    0.00   0.00   0.00
PERCENT    0.00   0.00   0.00
EXCHANGE   0.00   0.00   0.00
TICKER     0.00   0.00   0.00

